# Metro Manila Flood Level Predictor

This is a course requirement for the CS180 Artificial Intelligence Course of the Department of Computer Science, College of Engineering, University of the Philippines, Diliman under the guidance of Carlo Raquel for AY 2022-2023.

This is a Multilayer Perceptron model trained to predict the flood level in Metro Manila given the latitude, longitude, elevation, and precipitation rate.

## Importing libraries and dataset

The dataset that will be used is the [Metro Manila Flood Landscape Data](https://www.kaggle.com/datasets/giologicx/aegisdataset) uploaded by GiologicX on Kaggle. It has five features: the location separated into the latitude and longitude coordinates, the flood height, the elevation in meters, and the precipitation rate in millimeters per hour. As explained by the dataset creator, the data was extracted from different sources. The flood data was taken from flood reports by Project NOAH, the location and elevation data was retrieved from NAMRIA, and the precipitation data was averaged from data from PAGASA. The creator specified using Spatial Kriging to acquire the data.

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('AEGISDataset.csv')
data.head()

,lat,lon,flood_heig,elevation,precipitat
0,14.640394,121.055708,0,54.553295,9
1,14.698299,121.002132,0,21.856272,10
2,14.698858,121.100261,0,69.322807,16
3,14.571310,120.983334,0,10.987241,8
4,14.762232,121.075735,0,87.889847,18


## Preparing dataset for training and testing

The classifier requires a training set and a test set. Since the model will be predicting the flood height, then for each set, the column containing the flood height must be separated from the actual data. This also prevents the flood height from being affected by normalization.

70% of the dataset will be used for training while the remianing 30% will be used for testing.

In [49]:
x = data.drop('flood_heig', axis=1)
y = data['flood_heig']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,train_size=0.7, random_state=550)

## Normalizing the data

For the Multilayer Perceptron model to work optimally, the data is normalized into the [-1,1] range.

In [50]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1,1))

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Training and testing the model

The method that will be used for the model is Multilayer Perceptron, specifically, Multilayer Perceptron for Regression. According to a study by Mosavi et al. (2018), Multilayer Perceptron is one of the best and most popular machine learning methods for flood prediction. This method was chosen due to its simplicity, better ease of use, faster speed, and higher accuracy compared to other methods (Mosavi et al., 2018).

In [51]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(3),max_iter=1000)
mlp.fit(X_train,y_train)
predictions = mlp.predict(X_test)

## Results

We can evaluate our Multilayer Perceptron Regression model using the Root Mean Squared Error (RMSE) and the R Squared (R2). The lower the RMSE and the higher the R2 score, the better the model.

In [52]:
from sklearn.metrics import mean_squared_error
print("RMSE:",np.sqrt(mean_squared_error(y_test,predictions)))

from sklearn.metrics import r2_score
r2 = r2_score(y_test,predictions)
print("R2:",r2)

comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":predictions})
comparison_df

RMSE: 1.8027587894639463
R2: 0.04370311992156306


,Actual,Predicted
3127,5,2.697219
2454,3,2.331378
1710,2,2.120142
214,0,1.775570
704,0,2.142232
...,...,...
735,0,2.254512
2400,3,2.011554
3005,4,2.266195
1202,2,2.758273


Considering that the output of the model ranges from 0-8, 1.8 is a decently high RMSE, and 0.04 is quite a low R2 score. Thus, the model is not very good at predicting flood height.

## Hyperparameter Tuning

The model can be improved through hyperparameter tuning with GridSearchCV. Initial runs were done to determine the best solver, alpha, and activation values to use for the model. The run seen below is for determining the optimal values for the hidden layer sizes and initial learning rate.

In [53]:
param_grid = {'hidden_layer_sizes': [(3,2),(4,2),(5,2),(6,2),(7,2)], 
              'solver': ['adam'],
              'alpha': [0.001],
              'learning_rate_init': [0.1,0.01,0.001],
              'max_iter': [1000], 
              'activation': ['tanh']} 

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(MLPRegressor(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)
print(grid.best_params_)

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.1, max_iter=1000, solver=adam; total time=   0.3s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.1, max_iter=1000, solver=adam; total time=   0.2s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.1, max_iter=1000, solver=adam; total time=   0.2s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.1, max_iter=1000, solver=adam; total time=   0.2s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.1, max_iter=1000, solver=adam; total time=   0.2s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.01, max_iter=1000, solver=adam; total time=   1.3s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.01, max_iter=1000, solver=ada

C:\Users\Ieiaiel Sanceda\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   6.0s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   0.3s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   1.4s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   1.2s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(3, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   3.3s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(4, 2), learning_rate_init=0.1, max_iter=1000, solver=adam; total time=   0.4s
[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(4, 2), learning_rate_init=0.1, max_iter=1000, solver=adam; total time=   0.9s
[CV] END activation=tanh, alpha

C:\Users\Ieiaiel Sanceda\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(4, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   5.8s


...

[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(7, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   5.3s


C:\Users\Ieiaiel Sanceda\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(7, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   4.3s


C:\Users\Ieiaiel Sanceda\AppData\Roaming\Python\Python311\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV] END activation=tanh, alpha=0.001, hidden_layer_sizes=(7, 2), learning_rate_init=0.001, max_iter=1000, solver=adam; total time=   4.5s
{'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.01, 'max_iter': 1000, 'solver': 'adam'}


## Hyperparametrized model results

Once hyperparameter tuning is complete, we can evaluate the model using the same metrics used before.

In [54]:
grid_predictions = grid.predict(X_test)

print("RMSE:",np.sqrt(mean_squared_error(y_test,grid_predictions)))

r2 = r2_score(y_test,grid_predictions)
print("R2:",r2)

comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":grid_predictions})
comparison_df

RMSE: 1.7092277583686766
R2: 0.14035850632485547


,Actual,Predicted
3127,5,2.144595
2454,3,2.209193
1710,2,1.683344
214,0,1.105277
704,0,1.563703
...,...,...
735,0,2.397892
2400,3,1.133870
3005,4,2.797599
1202,2,3.384344


We can see that this new model yielded a lower RMSE and a higher R2 score compared to the untuned model. However, while the new model is better than the initial model, the scores are still unsatisfactory.

## Comparison with existing models

An existing notebook comparing a linear regression model and a geospatial weighted regression model can be found [here](https://www.kaggle.com/code/giologicx/geospatial-weighted-regression). The linear regression model yielded an adjusted R2 score of 0.005312, which is lower than our tuned MLP model. However, the geospatial weighted regression model resulted in a score of 0.2934473, which is better than the tuned MLP model.

## Conclusion

With the metrics used above, we can conclude that our model does not perform satisfactorily for the given dataset. The model with tuned hyperparameters yielded better results than the untuned model and the linear regression model in the existing notebook. However, the tuned model did not yield better results than the geospatial weighted regression model. The complex structure of geographical land, and the nature of flood height to vary greatly throughout a small interval of coordinates, might have been beyond what an MLP Regressor model could analyze. These shortcomings amight also be why the geospatial weighted regression model was created. Although previous studies have shown that MLP is one of the best models to use for flood prediction, the model is unoptimal for the chosen dataset.